In [1]:
import git, os, sys
git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")
os.chdir(f'{git_root}/src')
sys.path.append(os.path.abspath(os.path.join(f'{git_root}/src')))
print(f'Changed working directory to {os.getcwd()}')

Changed working directory to C:\Users\Alex\OneDrive\Documents\GitHub\UFC_Prediction_2022\src


In [2]:
#getting dependencies
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' (disables SettingWithCopyWarning)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
import itertools

# from fight_stat_helpers import *

In [ ]:
from sklearn.metrics import get_scorer_names

In [ ]:
print(get_scorer_names())

In [3]:
#scores a model
def model_score(dataframe, features, iloc_val = 3200, _max_iter = 2000, scoring='neg_log_loss', scaled=True):
    yyy=dataframe['result'].iloc[0:iloc_val]
    XXX=dataframe[features].iloc[0:iloc_val]
    XXXscaler = preprocessing.StandardScaler().fit(XXX)
    XXX_scaled = XXXscaler.transform(XXX) 
    X = XXX_scaled if scaled else XXX
    winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter)#, fit_intercept=False)
    # find the cross val score with log loss
    return cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()
    
#CODE FOR THE GREEDY ALGORITHM FOR FEATURE SELECTION
def greedy(dataframe, features, subsetsize, iloc_val=3200, _max_iter = 2000, scaled=True, scoring='neg_log_loss', set_of_sets=False):
    if set_of_sets:
        s=set([tuple(feature) for feature in features])
    else:
        s=set(features)
    subsets=list(map(set, itertools.combinations(s, subsetsize))) #subsets of size (subsetsize)
    scores_dict = {}
    for subset in subsets:
        if set_of_sets:
            list_of_features = []
            for feature_set in subset:
                list_of_features.extend(list(feature_set))
        else:
            list_of_features = list(subset)
        scores_dict[tuple(subset)]=model_score(dataframe, list_of_features, iloc_val, _max_iter, scaled=scaled, scoring=scoring)
    max_key = max(scores_dict, key=scores_dict.get)
    max_score = scores_dict[max_key]
    print(f'best subset: {max_key}')
    print(f'with score {max_score}')
    best_feature_list = []
    if set_of_sets:
        for feature_set in max_key:
            best_feature_list.extend(list(feature_set))
    else:
        best_feature_list = list(max_key)
    return best_feature_list, max_score

def reductive_greedy(dataframe, starting_features=None):
    # remove one at a time until it stops going up
    if starting_features is None:
        last_best_score = - 1000
        starting_features = dataframe.columns.tolist()
    else:
        last_best_score = model_score(dataframe, starting_features, _max_iter=5000)
        print(f'Starting with features: {starting_features}')
        print(f'Current best score: {last_best_score}')
    # remove result from list 
    if 'result' in starting_features:
        starting_features.remove('result')
    current_best_score = model_score(dataframe, starting_features, _max_iter=5000)
    features = starting_features.copy()
    while current_best_score > last_best_score:
        last_best_score = current_best_score
        new_features, current_best_score = greedy(dataframe, features, subsetsize=len(features)-1)
        features = new_features
        print(f'Current best score: {current_best_score}')
        print(f'Current best subset: {features}')
        
    print(f'No improvement found, stopping greedy search.')
    best_score = last_best_score
    print(f'Final best score: {best_score}')
    return features

def additive_greedy(dataframe, current_best_feature_set=[], search_doubles=False):
    # now start adding remaining features back in one by one or in groups of two or three and see if we can improve the score
    if current_best_feature_set:
        current_model_score = model_score(dataframe, current_best_feature_set, _max_iter=5000)
        print(f'Starting with current best feature set: {current_best_feature_set}')
        print(f'Current model score: {current_model_score}')
    else:
        print('Starting with an empty feature set.')
        current_model_score = -1000 
        
    while True:
        unused_features = set(dataframe.columns) - set(current_best_feature_set) - set(['result'])
        # first try adding one feature 
        single_feature_scores = {}
        for feature in unused_features:
            new_feature_set = current_best_feature_set + [feature]
            new_model_score = model_score(dataframe, new_feature_set, _max_iter=5000)
            single_feature_scores[tuple(new_feature_set)] = new_model_score
        best_single_feature_set = max(single_feature_scores, key=single_feature_scores.get)
        best_single_feature_score = single_feature_scores[best_single_feature_set]
        if best_single_feature_score > current_model_score:
            current_best_feature_set = list(best_single_feature_set)
            current_model_score = best_single_feature_score
            new_model_accuracy = model_score(dataframe, current_best_feature_set, _max_iter=5000, scoring='accuracy', scaled=False)
            print(f'Added single feature: {best_single_feature_set[-1]}')
            print(f'New model score: {current_model_score}')
            print(f'New model accuracy: {new_model_accuracy}')
            continue
        else:
            print('No improvement found with single addition, trying pairs.')
        if not search_doubles:
            print('Stopping search for pairs (doubles flag set to false in inputs).')
            break
        double_feature_scores = {}
        for feature1, feature2 in itertools.combinations(unused_features, 2):
            new_feature_set = current_best_feature_set + [feature1, feature2]
            new_model_score = model_score(dataframe, new_feature_set, _max_iter=5000)
            double_feature_scores[tuple(new_feature_set)] = new_model_score
        best_double_feature_set = max(double_feature_scores, key=double_feature_scores.get)
        best_double_feature_score = double_feature_scores[best_double_feature_set]
        if best_double_feature_score > current_model_score:
            current_best_feature_set = list(best_double_feature_set)
            current_model_score = best_double_feature_score
            print(f'Added double feature: {best_double_feature_set[-2]}, {best_double_feature_set[-1]}')
            print(f'New model score: {current_model_score}')
            continue
        else:
            print('No improvement found with double addition, stopping.')
            break
    return current_best_feature_set

In [15]:
ufc_fights_predictive_flattened_diffs_path = f'{git_root}/src/content/data/processed/ufc_fights_predictive_flattened_diffs.csv'
ufc_fights_predictive_flattened_diffs = pd.read_csv(ufc_fights_predictive_flattened_diffs_path)
ufc_fights_predictive_flattened_diffs = ufc_fights_predictive_flattened_diffs[::-1]

@np.vectorize
def clean_method_for_winner_predictions(a):
    if (a == 'KO/TKO'):
        return 'KO/TKO'
    elif (a == 'SUB'):
        return 'SUB'
    elif ((a == 'U-DEC') or (a == 'M-DEC')):
        return 'DEC'
    # counting S-DEC as bullshit!
    else:
        return 'bullshit'
    
#fights where the method of victory is TKO/SUB/DEC (no split decision or DQ or Overturned or anything else like that)
ufc_fights_predictive_flattened_diffs['method'] = clean_method_for_winner_predictions(ufc_fights_predictive_flattened_diffs['method'])

draw_mask=ufc_fights_predictive_flattened_diffs['result'] != 'D'

# drop any rows with nan in any column
ufc_fights_predictive_flattened_diffs=ufc_fights_predictive_flattened_diffs[draw_mask] #&method_mask_winner] # TODO add method column for cleaning purposes

# method bullshit mask
method_mask_winner = ufc_fights_predictive_flattened_diffs['method'] != 'bullshit'
ufc_fights_predictive_flattened_diffs = ufc_fights_predictive_flattened_diffs[method_mask_winner]


In [16]:
ufc_fights_predictive_flattened_diffs.head()

fighter       opponent result  method           division  \
8228  Tallison Teixeira  Derrick Lewis      L  KO/TKO        Heavyweight   
8226       Steve Garcia  Calvin Kattar      W     DEC      Featherweight   
8225   Morgan Charriere  Nate Landwehr      W  KO/TKO      Featherweight   
8224        Austen Lane  Vitor Petrino      L     SUB        Heavyweight   
8223        Tuco Tokkos    Junior Tafa      W     SUB  Light Heavyweight   

       age_diff    age_sum  age_diff_sq   age_sum_sq  height_diff  height_sum  \
8228 -14.828200  66.020534   219.875511  4358.710894          4.0       154.0   
8226  -4.156057  70.433949    17.272814  4960.941221          1.0       143.0   
8225  -7.386721  66.809035    54.563653  4463.447145         -1.0       137.0   
8224   9.801506  65.544148    96.069516  4296.035317          4.0       152.0   
8223   6.228611  63.838467    38.795589  4075.349844          1.0       151.0   

      height_diff_sq  height_sum_sq  reach_diff  reach_sum  reach_diff_sq  \
8228            16.0        23716.0         4.0      162.0           16.0   
8226             1.0        20449.0         3.0      147.0            9.0   
8225             1.0        18769.0        -3.0      141.0            9.0   
8224            16.0        23104.0         3.0      157.0            9.0   
8223             1.0        22801.0         4.0      148.0           16.0   

      reach_sum_sq  all_wins_diff  all_wins_sum  all_wins_diff_sq  \
8228       26244.0          -18.0          20.0             324.0   
8226       21609.0           -1.0          13.0               1.0   
8225       19881.0           -3.0           7.0               9.0   
8224       24649.0           -3.0           5.0               9.0   
8223       21904.0           -2.0           2.0               4.0   

      all_wins_sum_sq  l1y_wins_diff  l1y_wins_sum  l1y_wins_diff_sq  \
8228            400.0            1.0           1.0               1.0   
8226            169.0            2.0           2.0               4.0   
8225             49.0            1.0           1.0               1.0   
8224             25.0            1.0           1.0               1.0   
8223              4.0           -1.0           1.0               1.0   

      l1y_wins_sum_sq  l3y_wins_diff  l3y_wins_sum  l3y_wins_diff_sq  \
8228              1.0           -1.0           3.0               1.0   
8226              4.0            5.0           5.0              25.0   
8225              1.0           -1.0           5.0               1.0   
8224              1.0           -3.0           5.0               9.0   
8223              1.0           -2.0           2.0               4.0   

      l3y_wins_sum_sq  l5y_wins_diff  l5y_wins_sum  l5y_wins_diff_sq  \
8228              9.0           -4.0           6.0              16.0   
8226             25.0            4.0           8.0              16.0   
8225             25.0           -2.0           6.0               4.0   
8224             25.0           -3.0           5.0               9.0   
8223              4.0           -2.0           2.0               4.0   

      l5y_wins_sum_sq  all_wins_ko_diff  all_wins_ko_sum  all_wins_ko_diff_sq  \
8228             36.0             -14.0             16.0                196.0   
8226             64.0               2.0             10.0                  4.0   
8225             36.0               1.0              3.0                  1.0   
8224             25.0              -1.0              1.0                  1.0   
8223              4.0              -2.0              2.0                  4.0   

      all_wins_ko_sum_sq  l1y_wins_ko_diff  l1y_wins_ko_sum  \
8228               256.0               1.0              1.0   
8226               100.0               2.0              2.0   
8225                 9.0               1.0              1.0   
8224                 1.0               0.0              0.0   
8223                 4.0              -1.0              1.0   

      l1y_wins_ko_diff_sq  l1

In [17]:
ufc_fights_predictive_flattened_diffs = ufc_fights_predictive_flattened_diffs.dropna(axis=0, how='any')
y = (ufc_fights_predictive_flattened_diffs['result'] == 'W').values.astype(int)
# drop the 'fighter_result' column and opponent_result column as it is not needed for the model and also fighter and opponent columns
X = ufc_fights_predictive_flattened_diffs.drop(columns=['fighter', 'opponent', 'method', 'division'])
# KEEP result in X as this is what model_score is expecting for better or for worse
X['result'] = (X['result'] == 'W').astype(int)  # Convert result to binary (1 for win, 0 for loss)
# TODO filter by division
# only take the first 4000 rows for training
X = X.iloc[:4000, :]
y = y[:4000]

# try this later
# ufc_fights_predictive_doubled_path = f'{git_root}/src/content/data/processed/ufc_fights_predictive_doubled.csv'
# ufc_fights_predictive_doubled = pd.read_csv(ufc_fights_predictive_doubled_path)

In [18]:
ufc_fights_predictive_flattened_diffs.head()

fighter       opponent result  method           division  \
8228  Tallison Teixeira  Derrick Lewis      L  KO/TKO        Heavyweight   
8226       Steve Garcia  Calvin Kattar      W     DEC      Featherweight   
8225   Morgan Charriere  Nate Landwehr      W  KO/TKO      Featherweight   
8224        Austen Lane  Vitor Petrino      L     SUB        Heavyweight   
8223        Tuco Tokkos    Junior Tafa      W     SUB  Light Heavyweight   

       age_diff    age_sum  age_diff_sq   age_sum_sq  height_diff  height_sum  \
8228 -14.828200  66.020534   219.875511  4358.710894          4.0       154.0   
8226  -4.156057  70.433949    17.272814  4960.941221          1.0       143.0   
8225  -7.386721  66.809035    54.563653  4463.447145         -1.0       137.0   
8224   9.801506  65.544148    96.069516  4296.035317          4.0       152.0   
8223   6.228611  63.838467    38.795589  4075.349844          1.0       151.0   

      height_diff_sq  height_sum_sq  reach_diff  reach_sum  reach_diff_sq  \
8228            16.0        23716.0         4.0      162.0           16.0   
8226             1.0        20449.0         3.0      147.0            9.0   
8225             1.0        18769.0        -3.0      141.0            9.0   
8224            16.0        23104.0         3.0      157.0            9.0   
8223             1.0        22801.0         4.0      148.0           16.0   

      reach_sum_sq  all_wins_diff  all_wins_sum  all_wins_diff_sq  \
8228       26244.0          -18.0          20.0             324.0   
8226       21609.0           -1.0          13.0               1.0   
8225       19881.0           -3.0           7.0               9.0   
8224       24649.0           -3.0           5.0               9.0   
8223       21904.0           -2.0           2.0               4.0   

      all_wins_sum_sq  l1y_wins_diff  l1y_wins_sum  l1y_wins_diff_sq  \
8228            400.0            1.0           1.0               1.0   
8226            169.0            2.0           2.0               4.0   
8225             49.0            1.0           1.0               1.0   
8224             25.0            1.0           1.0               1.0   
8223              4.0           -1.0           1.0               1.0   

      l1y_wins_sum_sq  l3y_wins_diff  l3y_wins_sum  l3y_wins_diff_sq  \
8228              1.0           -1.0           3.0               1.0   
8226              4.0            5.0           5.0              25.0   
8225              1.0           -1.0           5.0               1.0   
8224              1.0           -3.0           5.0               9.0   
8223              1.0           -2.0           2.0               4.0   

      l3y_wins_sum_sq  l5y_wins_diff  l5y_wins_sum  l5y_wins_diff_sq  \
8228              9.0           -4.0           6.0              16.0   
8226             25.0            4.0           8.0              16.0   
8225             25.0           -2.0           6.0               4.0   
8224             25.0           -3.0           5.0               9.0   
8223              4.0           -2.0           2.0               4.0   

      l5y_wins_sum_sq  all_wins_ko_diff  all_wins_ko_sum  all_wins_ko_diff_sq  \
8228             36.0             -14.0             16.0                196.0   
8226             64.0               2.0             10.0                  4.0   
8225             36.0               1.0              3.0                  1.0   
8224             25.0              -1.0              1.0                  1.0   
8223              4.0              -2.0              2.0                  4.0   

      all_wins_ko_sum_sq  l1y_wins_ko_diff  l1y_wins_ko_sum  \
8228               256.0               1.0              1.0   
8226               100.0               2.0              2.0   
8225                 9.0               1.0              1.0   
8224                 1.0               0.0              0.0   
8223                 4.0              -1.0              1.0   

      l1y_wins_ko_diff_sq  l1

In [19]:
X.shape

(4000, 1213)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# NOTE A LARGER VALUE IS BETTER FOR NEG LOG LOSS

In [21]:
best_additive_features = additive_greedy(X_train, search_doubles=False) # 1215 too big for search doubles I think

Starting with an empty feature set.
Added single feature: age_diff
New model score: -0.6754070422316125
New model accuracy: 0.5875
Added single feature: l5y_wins_diff
New model score: -0.666208975250451
New model accuracy: 0.590625
Added single feature: l1y_inf_control_per_min_diff
New model score: -0.6620521280008667
New model accuracy: 0.5990624999999999
Added single feature: all_losses_diff
New model score: -0.6599460028631041
New model accuracy: 0.600625
Added single feature: all_abs_ground_strikes_accuracy_sum
New model score: -0.6583672730921284
New model accuracy: 0.60125
Added single feature: l3y_abs_ground_strikes_landed_per_min_diff
New model score: -0.6568214431995802
New model accuracy: 0.6046875
Added single feature: all_abs_takedowns_attempts_per_min_diff
New model score: -0.6549989684515645
New model accuracy: 0.6096874999999999
Added single feature: l3y_abs_clinch_strikes_landed_per_min_diff
New model score: -0.6536916633996789
New model accuracy: 0.6084375000000001
Add

c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://sci

Added single feature: l1y_inf_distance_strikes_attempts_per_min_sum_sq
New model score: -0.6281426744721091
New model accuracy: 0.6521875


c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://sci

Added single feature: l1y_inf_body_strikes_landed_per_min_diff
New model score: -0.6278973093319049
New model accuracy: 0.6581250000000001


...

Added single feature: all_abs_head_strikes_accuracy_sum
New model score: -0.6138575917643921
New model accuracy: 0.615


c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://sci

Added single feature: all_abs_takedowns_landed_per_min_diff
New model score: -0.6138245732755084
New model accuracy: 0.6209375


c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Alex\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 5000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=5000).
You might also want to scale the data as shown in:
    https://sci

Added single feature: all_wins_dec_diff
New model score: -0.6137722194614972
New model accuracy: 0.6174999999999999
No improvement found with single addition, trying pairs.
Stopping search for pairs (doubles flag set to false in inputs).


In [ ]:
len(best_additive_features)

In [ ]:
# remove one at a time until it stops going up
best_reductive_subset = reductive_greedy(X_train, starting_features=best_additive_features)

In [ ]:
best_model_score = model_score(X_train, best_reductive_subset, _max_iter=5000)
print(f'Best model score: {best_model_score}')
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')
accuracy = model_score(X_train, best_additive_features, _max_iter=5000, scoring='accuracy')
print(f'Accuracy of the model: {accuracy}')

In [ ]:
# train model on training set with best features 
best_model = LogisticRegression(solver='lbfgs', max_iter=5000)#, fit_intercept=False)
best_model.fit(X_train[best_additive_features], y_train)
# evaluate the model on the test set
test_score = best_model.score(X_test[best_additive_features], y_test)
print(f'Test set accuracy: {test_score}')

In [ ]:
# WOW THIS IS AMAZING
print('Best additive features:')
print('-------------------------')
print(f'Number of best additive features: {len(best_additive_features)}')
for col in best_additive_features:
    print(col)

In [ ]:
amazing_feature_set = [
    'age_diff',
    'l1y_inf_clinch_strikes_attempts_per_min_diff',
    'l5y_inf_head_strikes_accuracy_diff',
    'all_wins_diff',
    'l5y_abs_total_strikes_accuracy_diff',
    'all_abs_leg_strikes_accuracy_diff',
    'l2y_abs_leg_strikes_accuracy_diff',
    'l5y_inf_body_strikes_accuracy_diff',
    'l1y_abs_reversals_per_min_diff',
    'l2y_abs_clinch_strikes_accuracy_diff',
    'l1y_inf_leg_strikes_accuracy_diff',
    'all_wins_dec_diff',
    'all_inf_takedowns_accuracy_diff',
    'l1y_wins_ko_diff',
    'l1y_inf_ground_strikes_accuracy_diff',
    'l3y_inf_ground_strikes_accuracy_diff',
    'l1y_abs_distance_strikes_attempts_per_min_diff',
    'l1y_abs_body_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_landed_per_min_diff',
    'l3y_abs_takedowns_accuracy_diff',
    'all_abs_takedowns_accuracy_diff',
    'l3y_inf_distance_strikes_accuracy_diff',
    'l1y_offensive_standing_striking_score_diff',
    'l2y_wins_sub_diff',
    'all_abs_body_strikes_accuracy_diff',
    'l5y_abs_body_strikes_accuracy_diff',
    'l2y_defensive_grappling_loss_diff',
    'l3y_abs_body_strikes_accuracy_diff',
    'all_inf_knockdowns_per_min_diff',
    'l3y_defensive_grappling_loss_diff',
    'l2y_inf_head_strikes_attempts_per_min_diff',
    'l3y_inf_head_strikes_attempts_per_min_diff',
    'l1y_abs_total_strikes_landed_per_min_diff',
    'l3y_abs_leg_strikes_accuracy_diff',
    'l5y_abs_sig_strikes_accuracy_diff',
    'l2y_inf_sig_strikes_accuracy_diff',
    'l2y_inf_head_strikes_landed_per_min_diff',
    'l1y_abs_ground_strikes_accuracy_diff',
    'l1y_inf_sig_strikes_accuracy_diff',
    'l3y_inf_reversals_per_min_diff',
    'all_inf_reversals_per_min_diff',
    'l2y_wins_ko_diff',
    'l1y_inf_head_strikes_landed_per_min_diff',
    'l2y_inf_ground_strikes_accuracy_diff',
    'l3y_abs_takedowns_attempts_per_min_diff',
    'all_abs_takedowns_landed_per_min_diff',
    'l1y_abs_takedowns_attempts_per_min_diff',
    'l5y_inf_takedowns_attempts_per_min_diff',
    'l3y_inf_takedowns_attempts_per_min_diff',
    'l5y_inf_knockdowns_per_min_diff',
    'l3y_num_fights_diff',
    'l1y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_inf_clinch_strikes_accuracy_diff',
    'l3y_inf_distance_strikes_attempts_per_min_diff',
    'l2y_num_fights_diff',
    'l5y_inf_ground_strikes_accuracy_diff',
    'all_abs_head_strikes_landed_per_min_diff',
    'l5y_abs_head_strikes_landed_per_min_diff',
    'l1y_abs_sig_strikes_attempts_per_min_diff',
    'l3y_abs_head_strikes_attempts_per_min_diff',
    'l5y_abs_takedowns_landed_per_min_diff',
    'l5y_inf_leg_strikes_attempts_per_min_diff',
    'l3y_inf_leg_strikes_landed_per_min_diff',
    'l1y_inf_distance_strikes_attempts_per_min_diff',
    'l1y_wins_dec_diff',
    'l3y_losses_dec_diff',
    'l2y_losses_ko_diff',
    'l5y_losses_ko_diff',
    'all_inf_leg_strikes_attempts_per_min_diff',
    'l5y_inf_leg_strikes_accuracy_diff',
    'l3y_inf_clinch_strikes_landed_per_min_diff',
    'l5y_inf_clinch_strikes_landed_per_min_diff',
    'l1y_abs_clinch_strikes_landed_per_min_diff',
    'l3y_abs_clinch_strikes_landed_per_min_diff',
    'l1y_abs_clinch_strikes_accuracy_diff',
    'l1y_abs_ground_strikes_attempts_per_min_diff',
    'l5y_abs_clinch_strikes_landed_per_min_diff',
]

In [ ]:
# try removing features again 
best_reductive_subset = reductive_greedy(X, starting_features=amazing_feature_set)

In [ ]:
# probability of observing data given the model
best_model_score = model_score(X, best_reductive_subset, _max_iter=5000)
print(f'Probability to observe data given the model: {np.exp(best_model_score)}')

In [ ]:
X.shape

In [ ]:
# get accuracy of model
accuracy = model_score(X, amazing_feature_set, _max_iter=5000, scoring='accuracy')
print(f'Accuracy of the model: {accuracy}')

# WHY IS THE SCORE DIFFERENT THAN FROM ABOVE???

In [ ]:
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    # TODO rescale the features to zero mean and unit variance
    
    # return [fighter_score_diff(fighter1,fighter2,day1, 4),
    #         fighter_score_diff(fighter1,fighter2,day1, 9),
    #         fighter_score_diff(fighter1,fighter2,day1, 15),
    #         fight_math_diff(fighter1,fighter2,day1, 1),
    #         fight_math_diff(fighter1,fighter2,day1, 6),
    #         L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
    #         L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
    #         L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
    #         fighter_age(fighter1,day1)-fighter_age(fighter2,day2),
    #         avg_count('total_strikes_landed',fighter1,'abs',day1)-avg_count('total_strikes_landed',fighter2,'abs',day2),
    #         avg_count('head_strikes_landed',fighter1,'abs',day1)-avg_count('head_strikes_landed',fighter2,'abs',day2),
    #         avg_count('ground_strikes_landed',fighter1,'inf',day1)-avg_count('ground_strikes_landed',fighter2,'inf',day2),
    #         avg_count('takedowns_attempts',fighter1,'inf',day1)-avg_count('takedowns_attempts',fighter2,'inf',day2),
    #         avg_count('head_strikes_landed',fighter1,'inf',day1)-avg_count('head_strikes_landed',fighter2,'inf',day2),
    #     ]
    return [
    # 'fighter_age_diff',
    #     'height_diff',
    #     'reach_diff',
    #     'fighter_L5Y_wins_diff_2',
    #     'fighter_L5Y_losses_diff_2',
    #     'fighter_ko_losses_diff_2',
    #     'fighter_L5Y_ko_wins_diff_2',
    #     'fighter_L2Y_ko_losses_diff_2',
        fighter_age(fighter1, day1) - fighter_age(fighter2, day2),
        fighter_height(fighter1) - fighter_height(fighter2),
        fighter_reach(fighter1) - fighter_reach(fighter2),
        L5Y_wins(fighter1, day1) - L5Y_wins(fighter2, day2),
        L5Y_losses(fighter1, day1) - L5Y_losses(fighter2, day2),
        ko_losses(fighter1, day1) - ko_losses(fighter2, day2),
        L5Y_ko_wins(fighter1, day1) - L5Y_ko_wins(fighter2, day2),
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        #     'fighter_L5Y_ko_losses_diff_2',
    #     'fighter_sub_wins_diff_2',
    #     'fighter_L2Y_sub_losses_diff_2',
    #     'fighter_inf_knockdowns_avg_diff_2',
        L2Y_ko_losses(fighter1, day1) - L2Y_ko_losses(fighter2, day2),
        sub_wins(fighter1, day1) - sub_wins(fighter2, day2),
        L2Y_sub_losses(fighter1, day1) - L2Y_sub_losses(fighter2, day2),
        avg_count('knockdowns', fighter1, 'inf', day1) - avg_count('knockdowns', fighter2, 'inf', day2),
    
    #     'fighter_inf_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'inf', day1) - avg_count('distance_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_ground_strikes_landed_avg_diff_2',
        avg_count('ground_strikes_landed', fighter1, 'inf', day1) - avg_count('ground_strikes_landed', fighter2, 'inf', day2),
    #     'fighter_inf_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'inf', day1) - avg_count('distance_strikes_attempts', fighter2, 'inf', day2),
        
        
        #     'fighter_inf_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'inf', day1) - avg_count('head_strikes_attempts', fighter2, 'inf', day2),
    #     'fighter_inf_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'inf', day1) - avg_count('takedowns_attempts', fighter2, 'inf', day2),
    #     'fighter_abs_reversals_avg_diff_2',
        # avg_count('reversals', fighter1, 'abs', day1) - avg_count('reversals', fighter2, 'abs', day2),
    #     'fighter_abs_clinch_strikes_landed_avg_diff_2',
        avg_count('clinch_strikes_landed', fighter1, 'abs', day1) - avg_count('clinch_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_landed_avg_diff_2',
        avg_count('distance_strikes_landed', fighter1, 'abs', day1) - avg_count('distance_strikes_landed', fighter2, 'abs', day2),
        
            #     'fighter_abs_leg_strikes_landed_avg_diff_2',
        avg_count('leg_strikes_landed', fighter1, 'abs', day1) - avg_count('leg_strikes_landed', fighter2, 'abs', day2),
    #     'fighter_abs_takedowns_attempts_avg_diff_2',
        avg_count('takedowns_attempts', fighter1, 'abs', day1) - avg_count('takedowns_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_distance_strikes_attempts_avg_diff_2',
        avg_count('distance_strikes_attempts', fighter1, 'abs', day1) - avg_count('distance_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_head_strikes_attempts_avg_diff_2',
        avg_count('head_strikes_attempts', fighter1, 'abs', day1) - avg_count('head_strikes_attempts', fighter2, 'abs', day2),
    #     'fighter_abs_total_strikes_attempts_avg_diff_2',
        avg_count('total_strikes_attempts', fighter1, 'abs', day1) - avg_count('total_strikes_attempts', fighter2, 'abs', day2),
                #     '6-fight_math',
    #     '9-fighter_score_diff',
    #     '4-fighter_score_diff',
        fight_math_diff(fighter1, fighter2, day1, 6),
        fighter_score_diff(fighter1, fighter2, day1, 9),
        fighter_score_diff(fighter1, fighter2, day1, 4),
    ]
    

In [ ]:
dataframe = ufc_fights_winner
iloc_val = 3200
_max_iter = 2000
scoring='neg_log_loss'
scaled=True

yyy=dataframe['result'].iloc[0:iloc_val]
XXX=dataframe[current_best_feature_set].iloc[0:iloc_val]
XXXscaler = preprocessing.StandardScaler().fit(XXX)
XXX_scaled = XXXscaler.transform(XXX) 
X = XXX_scaled if scaled else XXX
winPredictionModel=LogisticRegression(solver='lbfgs', max_iter=_max_iter)#, fit_intercept=False)
# find the cross val score with log loss
cross_val_score(winPredictionModel,X,yyy,cv=4,scoring=scoring).mean()

In [ ]:
tup = ufc_prediction_tuple('Cody Garbrandt', 'Pedro Munhoz', 'July 5, 2023', 'July 5, 2023')

In [ ]:
tup_scaled = XXXscaler.transform(np.array(tup).reshape(1, -1))

In [ ]:
tup_scaled[0]